In [1]:
import numpy as np
import pandas as pd
from pyecharts.globals import CurrentConfig, NotebookType
from pyecharts import options as opts
from pyecharts.charts import Bar

# 配置 Pyecharts
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

In [2]:
data1 = pd.read_excel('超市.xlsx')
data1.head(1)

,行 ID,订单 ID,订单日期,发货日期,邮寄方式,客户 ID,客户名称,细分,城市,省/自治区,国家,地区,产品 ID,类别,子类别,产品名称,销售额,数量,折扣,利润
0,1,US-2018-1357144,2018-04-27,2018-04-29,二级,曾惠-14485,曾惠,公司,杭州,浙江,中国,华东,办公用-用品-10002717,办公用品,用品,"Fiskars 剪刀, 蓝色",129.696,2,0.4,-60.704


### 需求：分析一下各省的销售额分布情况

###### 取数

In [3]:
data2 = data1.groupby('省/自治区')['销售额'].sum()
# 还是那理由
data2 = pd.DataFrame(data2)
data2.head(1)

,销售额
省/自治区,
上海,582450.568


In [4]:
data2.head(1)

,销售额
省/自治区,
上海,582450.568


In [5]:
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.faker import Faker

# 第一个值
x = data2.index.tolist()
# 第二个值
y = data2['销售额'].tolist()

c = (
    Geo()
    .add_schema(maptype="china")     # 👇 添加到这
    .add("geo", [list(z) for z in zip(x, y)])
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        visualmap_opts=opts.VisualMapOpts(), title_opts=opts.TitleOpts(title="Geo-基本示例")
    )
)
c.render_notebook()

In [6]:
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.faker import Faker
from pyecharts.globals import ChartType

c = (
    Geo()
    .add_schema(maptype="china")
    .add(
        "geo",
        [list(z) for z in zip(x, y)],
        type_=ChartType.HEATMAP,
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        visualmap_opts=opts.VisualMapOpts(),
        title_opts=opts.TitleOpts(title="Geo-HeatMap"),
    )
)
c.render_notebook()

#### 雷达图

#### 需求：查看一下各地区细分的销售额

In [7]:
data3 = data1.groupby(['地区','细分'])['销售额'].sum()
data3 = pd.DataFrame(data3).unstack()
data3 

销售额                         
细分           公司        小型企业          消费者
地区                                      
东北   834842.827  522739.035  1323985.607
中南  1335665.324  743813.007  2057936.762
华东  1454715.808  942432.372  2287358.262
华北   804769.469  422100.987  1220430.561
西北   253458.184  103523.308   458058.104
西南   469341.684  156479.932   677302.892

In [8]:
# 取三个数据
data_1 = [data3['销售额']['公司'].tolist()]
data_2 = [data3['销售额']['小型企业'].tolist()]
data_3 = [data3['销售额']['消费者'].tolist()]
data_3

[[1323985.607, 2057936.762, 2287358.262, 1220430.561, 458058.104, 677302.892]]

In [9]:
import pyecharts.options as opts
from pyecharts.charts import Radar

"""
Gallery 使用 pyecharts 1.1.0
参考地址: https://echarts.apache.org/examples/editor.html?c=radar

目前无法实现的功能:

1、雷达图周围的图例的 textStyle 暂时无法设置背景颜色
"""
v1 = data_1
v2 = data_2
v3 = data_3

c = (
    Radar(init_opts=opts.InitOpts(bg_color="#CCCCCC"))
    .add_schema(
        schema=[
            opts.RadarIndicatorItem(name="东北", max_=2000000),
            opts.RadarIndicatorItem(name="中南", max_=2000000),
            opts.RadarIndicatorItem(name="华东", max_=2000000),
            opts.RadarIndicatorItem(name="华北", max_=2000000),
            opts.RadarIndicatorItem(name="西北", max_=2000000),
            opts.RadarIndicatorItem(name="西南", max_=2000000),
        ],
        splitarea_opt=opts.SplitAreaOpts(
            is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
        ),
        textstyle_opts=opts.TextStyleOpts(color="#fff"),
    )
    .add(
        series_name="预算分配（Allocated Budget）",
        data=v1,
        linestyle_opts=opts.LineStyleOpts(color="#CD0000"),
    )
    .add(
        series_name="实际开销（Actual Spending）",
        data=v2,
        linestyle_opts=opts.LineStyleOpts(color="#5CACEE"),
    )
    .add(
        series_name="分配（Allocated Budget）",
        data=v3,
        linestyle_opts=opts.LineStyleOpts(color="#CD0000"),
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="基础雷达图"), legend_opts=opts.LegendOpts()
    )
)
c.render_notebook()


### 漏斗图

#### 分析用户行为

In [10]:
data4 = pd.read_excel('./漏斗.xlsx')
data4

,用户主要行为,用户数
0,DAU,700000
1,登录,500000
2,加入购物车,300000
3,生成订单,210000
4,支付,150000
5,完成交易,140000


In [11]:
from pyecharts import options as opts
from pyecharts.charts import Funnel
from pyecharts.faker import Faker

# 取数
x = data4['用户主要行为']
y = data4['用户数']

lou = (
    Funnel()                       
    .add(
        "商品",              #  👇 放这
        [list(z) for z in zip(x,y)],
        # 👇 这个使 标签 放 内部
        label_opts = opts.LabelOpts(position = "inside")
    ) 
    .set_global_opts(title_opts=opts.TitleOpts(title="分析用户行为"))
)
lou.render_notebook()

#### 象形图

#### 需求：不同地区销售额对比

In [12]:
df1 = data1.groupby('地区')['销售额'].sum()
df1 = pd.DataFrame(df1)
df1

,销售额
地区,
东北,2681567.469
中南,4137415.093
华东,4684506.442
华北,2447301.017
西北,815039.596
西南,1303124.508


In [13]:
# 取数
location = df1.index.to_list()
values =df1['销售额'].to_list()

In [14]:
from pyecharts import options as opts
from pyecharts.charts import PictorialBar
from pyecharts.globals import SymbolType

c = (
PictorialBar()  # 👇第一个参数
    .add_xaxis(location)
    .add_yaxis(
        "",
        # 👇 第二个参数
        values,
        label_opts=opts.LabelOpts(is_show=False),
        symbol_size=18,
        symbol_repeat="fixed",
        is_symbol_clip=True,
        symbol=SymbolType.ROUND_RECT
    )                     # 👆 这个是形状
    .reversal_axis()
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各地区销售额分布"),
        xaxis_opts=opts.AxisOpts(is_show=False),
        yaxis_opts=opts.AxisOpts()
    )
)
c.render_notebook()

#### 词云图

##### 需求：分析热点省份

In [15]:
data1.columns

Index(['行 ID', '订单 ID', '订单日期', '发货日期', '邮寄方式', '客户 ID', '客户名称', '细分', '城市',
       '省/自治区', '国家', '地区', '产品 ID', '类别', '子类别', '产品名称', '销售额', '数量', '折扣',
       '利润'],
      dtype='object')

In [16]:
data6 = data1.groupby('省/自治区')['销售额'].sum()
data6 = pd.DataFrame(data6)

In [17]:
# 取数
x_data = data6.index.tolist()
y_data = data6['销售额'].tolist()

In [18]:
# 给数整形
list_ = []
for i in range(len(x_data)):
    list_.append((x_data[i],y_data[i]))

In [19]:
import pyecharts.options as opts
from pyecharts.charts import WordCloud
from pyecharts.globals import ThemeType

ci = (
    WordCloud(init_opts=opts.InitOpts(theme = ThemeType.VINTAGE))                       # 👇 列表放这里
    .add(series_name="热点分析", data_pair=list_, word_size_range=[6, 66])
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="热点分析", title_textstyle_opts=opts.TextStyleOpts(font_size=23)
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    )
)
ci.render_notebook()

In [20]:
from pyecharts import options as opts
from pyecharts.charts import Bar, Line, Grid

# 创建柱状图
bar = (
    Bar()
    .add_xaxis(["A", "B", "C", "D", "E"])
    .add_yaxis("Series A", [10, 20, 30, 40, 50])
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Bar Chart",pos_left="10%",pos_bottom="35%"),
        legend_opts=opts.LegendOpts(pos_left="15%"))
    )

# 创建折线图
line = (
    Line()
    .add_xaxis(["A", "B", "C", "D", "E"])
    .add_yaxis("Series B", [50, 40, 30, 20, 10])
    .set_global_opts(title_opts=opts.TitleOpts(title="Line Chart",pos_left="10%",pos_top="5%"),
                     legend_opts=opts.LegendOpts(pos_left="5%"))
)

# 创建 Grid 对象，并将柱状图和折线图添加到 Grid 对象中
grid = Grid(init_opts=opts.InitOpts(width="1000px", height="500px"))
grid.add(line, grid_opts=opts.GridOpts(pos_left="10%", pos_right="10%", pos_top="15%", height="40%"))
grid.add(bar, grid_opts=opts.GridOpts(pos_left="10%", pos_right="10%", pos_top="65%", height="25%"))

# 渲染图表
grid.render_notebook()

In [21]:
from pyecharts import options as opts
from pyecharts.charts import BMap
from pyecharts.faker import Faker

c = (
    BMap()
    .add_schema(baidu_ak="53oVIOgmSIejwV7EfphPgTynOZbIiVYu", center=[120.13066322374, 30.240018034923])
    .add(
        "bmap",
        [list(z) for z in zip(Faker.provinces, Faker.values())],
        label_opts=opts.LabelOpts(formatter="{b}"),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="BMap-基本示例"))
)
c.render_notebook()

In [22]:
data8 = pd.read_excel('./箱形图2.xlsx')
data8

,电脑品牌,IBM,联想,戴尔,小米,宏碁,华为
0,2017Q4,770,566,222,344,343,110
1,2018Q1,450,700,778,346,554,99
2,2018Q2,775,800,554,477,223,130
3,2018Q3,600,690,555,576,530,200
4,2018Q4,732,760,600,431,660,323
5,2019Q1,763,573,621,440,539,353
6,2019Q2,821,775,480,616,300,400
7,2019Q3,763,840,721,570,340,461
8,2019Q4,654,750,600,521,367,536


In [23]:
from pyecharts import options as opts
from pyecharts.charts import Boxplot

v1 = [
    data8['IBM'].tolist(),
    data8['联想'].tolist(),
    data8['戴尔'].tolist(),
    data8['小米'].tolist(),
    data8['宏碁'].tolist(),
    data8['华为'].tolist()
]

c = Boxplot()
c.add_xaxis(["IBM", "联想", "戴尔", "小米", "宏碁", "华为"])
c.add_yaxis("品牌", c.prepare_data(v1))
c.set_global_opts(title_opts=opts.TitleOpts(title="BoxPlot-基本示例"))
c.render_notebook()

In [24]:
from pyecharts import options as opts
from pyecharts.charts import Gauge

c = (
    Gauge()
    .add("", [("完成率", 66.6)])
    .set_global_opts(title_opts=opts.TitleOpts(title="Gauge-基本示例"))
)
c.render_notebook()

In [25]:
data9 = pd.DataFrame({'目标':[1000000],'完成':[680000]})
data9

,目标,完成
0,1000000,680000


In [26]:
rate = data9['完成']/data9['目标']
rate = rate*100
rate

0    68.0
dtype: float64

In [27]:
from pyecharts import options as opts
from pyecharts.charts import Gauge

c = (
    Gauge()
    .add("", [("完成率", rate)])
    .set_global_opts(title_opts=opts.TitleOpts(title="Gauge-基本示例"))
)
c.render_notebook()